# Database

## Relational Database Management System (RDBMS)

- RDBMS is the most straightforward DB system (MySQL etc)
- In every RDBMS, you want to have these **ACID** properties for a database transaction:
    - Atomicity: Every transaction is all or nothing
    - Consistency: Any transaction takes a database from one valid state to another valid state
    - Isolation: Executing transactions concurrently leads to the same result as executing them serially
    - Durability: Once a transaction is committed, it remains committed

### Scaling a relational DB

- Replication 
    - Master-slave replication
        - How does it work?
            - You have a few DB replicas, and designate 1 as master. 
            - The master can read and write, and slaves can only read
            - If a master goes offline, a slave is promoted to the master
            - Master replicates to slaves, and slaves can replicate to each other recursively
        
        - Disadvanges:
            - You need extra logic to deal with the slave promotion when master goes offline

    - Master-master replication
        - Similar to master-slave, but all replicas can handle read and writes
        
        - Disadvantages;
            - You'll need a load balancer or you'll need to define where to write in your code somewhere
            - Master-master are are either loosely consistent (because you must allow for DBs to exist in different states when multiple writes are happening) or have increased write latency due to synchronization.
            - Conflict resolution can add latency if you have more "write" nodes

    - General issues with replication:
        - If your master fails before the data can be replicated to other replicas, you will end up losing data
        - For every write, you need to "duplicate" the transaction to all replicas. So write-heavy systems will be extremely bogged down by constant writing and may not be as responsive with the reads
        - The more "read" slaves you have, the more replication has to take place, which can lag the entire DB system
        - Some systems may support multi-threaded writing to the master, but read replicas generally only support single threaded writes.
        - Replication adds hardware requirements and can be complex to manage

- Federation
    - What is it?
        - This just means splitting your DB up by function (i.e. different schemas/tables) instead of a single DB monolith
    
    - Advantages
        - Less read/writes into each DB, means less replication lag
        - Smaller data means data fits in memory, which lets more queries hit the cache, reducing expensive disk retrieval
        - If each DB can handle its own writes, then there is wider throughput, because you can write to multiple tables at once instead of doing writes serially

    - Disadvantages
        - Federation assumes all your tables/functions are roughly the same size. But if you have a few disproportionately large ones, then this not effective because a lot of the reads/writes into the massive DB will still be bottlenecked
        - Depending on which DB you want to update, you need to change your application logic
        - Joining tables is a lot more complex, because it requires a network / server link 
        - Adds complexity

- Sharding
    - What is it?
        - Like federation, you split your DB. But unlike federation, that splits the DB by function, sharding lets you split it by some data subset (i.e. all users with names starting with A goes to DB 1, B goes to DB 2 etc)

    - Advantages
        - Like federation, this lets you have better throughput (assuming your reads/writes are distributed evenly across all DBs)
        - If one shard goes down, the others are still up

    - Disadvantages
        - Your application logic needs to work around the DB shards, which may be a pain
        - Data distribution can become uneven, whcih reduces the efficiency of sharding
            - Rebalancing may be hard, and will affect legacy code
        - Joins become complex
        - Added hardware

 - Denormalization
    - What is it?
        - Usually, you want to "normalise" your tables as much as possible (i.e. every table to contain as little information as possible)
        - Idea being, you spend as little on storage as possible
        - But if you have joins that are constantly being done, the compute costs may outweigh the storage cost
        - So sometimes, storing "denormalised" tables may be cheaper!
        - Especially if tables are read heavy

    - Disadvantages
        - Data is duplicated, so space is wasted
        - With denormalised data, you need to make use of DB constraints to ensure that redundant copies of information stay in sync with other tables (e.g. if customer account is deleted from users table, then transactions must also be deleted)
        - A denormalized database under heavy write load might perform worse than its normalized counterpart.

- SQL tuning
    - What is it?
        - Basically writing sql properly for faster querying
        - Approaches
            - Tighten up schema
                - `CHAR` instead of `VARCHAR` for fixed length strings
                - `TEXT` for large text blocks
                - `INT`/`BIGINT` and `FLOAT`/`DOUBE` etc
            - Use better indices
            - Avoid expensive joins
            - Partition queries
            - Tune query cache

## NoSQL

- Data items represented in a key-value store / document store / wide column store / graph database. 
- Data is denormalized (because it is a key value store lol), and usually any joins are handled in the application code. 
- NoSQL stores lack true ACID transactions and favor eventual consistency (to ensure partition tolerance and availability)

- NoSQL DBs have the **BASE** concept instead of **ACID**
    - Basically Available: Guarantees availability
    - Soft State: System state may change over time even without input
    - Eventual Consistency: System will eventually become consistent over a period of time so long as it doesn't receive input

### Types of NoSQL DBs

- Key-Value Store
    - Abstraction is a hash table
    - Guarantees O(1) lookup, so use for read heavy applications.
    - Usually used for in-memory caches (i.e. Redis)
    - Usually forms the basis of more complex systems (like document stores or graph DBs)

- Document Store
    - Abstraction is a key-value store
    - Document stores all information for a given object
    - Document stores provide APIs or a query language to query based on the internal structure of the document itself
    - Each document can have completely different fields from each other
    - MongoDB, CouchDB, DDB
    - Used for working with occasionally changing data.

- Wide Column Store
    - Abstraction: nested map ColumnFamily<RowKey, Columns<ColKey, Value, Timestamp>>
    - Structure
        - basic unit of data is a column (name-value pair)
        - Each column can be grouped into column families
        - Each column family can be grouped into super column families
    - You can access each column independently with a row key, and columns with the same row key form a row
    - Usually used for cases where reads and writes are super high frequency
        - For example, high frequency sensor data
        - Each sensor is a row, each column family can be temperature, humidity etc
        - Each column can be a microsecond

- Graph database
    - Abstraction: graph
    - Each node is a record/entity, each edge a relationship
    - Usually can only be accessed via REST APIs






## SQL vs NoSQL

- Use SQL if:
    - Structured data
    - Strict schema
    - Relational data
    - Need for complex joins
    - Transactions
    - Clear patterns for scaling
    - More established: developers, community, code, tools, etc
    - Lookups by index are very fast

- Use NoSQL if:
    - Rapid ingest of clickstream and log data
    - Leaderboard or scoring data
    - Temporary data, such as a shopping cart
    - Frequently accessed ('hot') tables
    - Metadata/lookup tables